In [ ]:
import numpy as np
import cv2 as cv
import datetime
from sklearn import cluster,datasets
from scipy.stats import multivariate_normal
from sklearn.cluster import KMeans
from datetime import datetime, timedelta
import csv
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from numpy.linalg import inv
from sklearn.gaussian_process.kernels import RBF
from sklearn.metrics import accuracy_score
from sklearn.gaussian_process import GaussianProcessClassifier

In [ ]:
def file_reading(file_path):
    #reading annotations file
    ann_file = open(file_path,"r") #opening file in read mode only
    strings = [x.strip() for x in ann_file.readlines()]
    stimes=[]
    etimes=[]
    for i in range(len(strings)):
        s1,s2=strings[i].split("-")
        stimes.append(s1.strip())
        etimes.append(s2.strip())
    return stimes,etimes

In [ ]:
def compare(stime,etime,abnormal_stimes,abnormal_etimes):
    length = len(abnormal_stimes)
    for i in range(length):
        t1 = datetime.strptime(abnormal_stimes[i], '%M:%S').time()
        t2 = datetime.strptime(abnormal_etimes[i], '%M:%S').time()
        obj1 = timedelta(hours=t1.hour, minutes=t1.minute, seconds=t1.second)
        obj2 = timedelta(hours=t2.hour, minutes=t2.minute, seconds=t2.second)
        if (stime >= obj1 and etime <= obj2) or (stime < obj1 and etime > obj1) or (stime < obj2 and etime > obj2):
            return 1
    return -1

In [ ]:
class labeling_objects:
    def __init__(self,clip_no,stime,etime,label):
        self.clip_no = clip_no
        self.stime = stime
        self.etime = etime
        self.label = label

In [ ]:
def video_input(path,ann_file_path):
    cap = cv.VideoCapture(path)
    ret, frame1 = cap.read()

    #reading first frame
    prvs_gray = cv.cvtColor(frame1,cv.COLOR_BGR2GRAY)
    width=prvs_gray.shape[1]
    height = prvs_gray.shape[0]

    prvs = cv.resize(prvs_gray,(int(width/10),int(height/10)),interpolation = cv.INTER_AREA)

    #intializing all values
    cnt=0
    flow_array = [] # to store output from farneback
    #mag_list = []
    flow_array_array = [] #for storing all clips
    secs = 0
    clip = 0
    label_objects_array = []
    abnormal_stimes,abnormal_etimes = file_reading(ann_file_path)
    
    while(True):
        ret, frame2 = cap.read()
        cnt=cnt+1
        if cnt%50 == 0:
            flow_array_array.append(flow_array)
            flow_array=[]
            clip+=1
            #adding labels here after one clip is recorded.
            secs = secs+2
            stime = timedelta(seconds = secs-2)
            etime = timedelta(seconds = secs)
            label = compare(stime,etime,abnormal_stimes,abnormal_etimes)
            label_objects_array.append(labeling_objects(clip,stime,etime,label))
            
        if ret==False:
            break

        #converting frame into gray and resizing 
        gray2 = cv.cvtColor(frame2,cv.COLOR_BGR2GRAY)
        next = cv.resize(gray2,(int(width/10),int(height/10)),interpolation = cv.INTER_AREA)

        #calculating optical flow giving two consecutive frames as input
        flow = cv.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)

        # appending to the flow array
        flow_array.append(flow)

        #changing current frame as previous frame
        prvs = next
    if len(flow_array)!= 0:
        flow_array_array.append(flow_array)
        clip+=1
        secs = secs+1
        stime = timedelta(seconds = secs-1)
        etime = timedelta(seconds = secs)
        label = compare(stime,etime,abnormal_stimes,abnormal_etimes)
        label_objects_array.append(labeling_objects(clip,stime,etime,label))
    flow_array_length = len(flow_array_array)
    print("Total no of clips",flow_array_length)
    print("total frames",cnt)
    
    return flow_array_array,label_objects_array

In [ ]:
class GMM:
    
    def __init__(self,X,k,weights,means,variances,n_iter):
        self.X=X
        self.k=k
        self.weights=weights
        self.means = means
        self.variances = variances
        self.eps=1e-8
        self.n_iter = n_iter
        
    def run(self):
        for step in range(self.n_iter):
        
            likelihood=[]
            for j in range(self.k):
                likelihood.append(self.pdf(self.X, self.means[j], np.sqrt(self.variances[j])))
            likelihood = np.array(likelihood)
            
            b = []
            # Maximization step 
            
            for j in range(self.k):
                # use the current values for the parameters to evaluate the posterior
                # probabilities of the data to have been generanted by each gaussian    
                b.append((likelihood[j] * self.weights[j]) / (np.sum([likelihood[i] * self.weights[i] for i in range(self.k)],axis=0))+self.eps)

                # updage mean and variance
                self.means[j] = np.sum(b[j] * self.X) / (np.sum(b[j]+self.eps))
                self.variances[j] = np.sum(b[j] * np.square(self.X - self.means[j])) / (np.sum(b[j]+self.eps))
                
                #print("b dist=",b[j])
                # update the weights
                self.weights[j] = np.mean(b[j])
            #print(self.weights)
            return self

    def pdf(self,data,mean:float,variance:float):
        s1 = 1/(np.sqrt(2*np.pi*variance))
        s2 = np.exp(-(np.square(data - mean)/(2*variance)))
        return s1*s2

In [ ]:
#Function for distribution formation from the mag array of a clip
def func_distribution_formation(arr):
    dist_arr =[] 
    height,width,x =arr[0].shape
    for idx in range(int(height)):
        for j in range(int(width)):
            dist=[]
            for i in range(len(arr)):
                dist.append(arr[i][idx,j])
            dist=np.asarray(dist)
            dist_arr.append(dist)
    dist_arr=np.asarray(dist_arr)
    return dist_arr

In [ ]:
#intial parameters for em clustering 
def intial_parameters(arr):
    k=4
    arr = np.asarray(arr)
    intial_weights = [(1-0.1)/k for i in range(k)]
    intial_weights = np.append(intial_weights,0.1) # adding extra weight element for background purpose
    means = np.random.choice(arr.flatten(),(k,2))
    means =np.append(means,[0,0]) # adding extra mean element for background purpose

    cov = np.random.sample(size=k)
    cov = np.append(cov,4.0) # adding extra cov element for background purpose
    return k,intial_weights,means,cov

In [ ]:
class data_objects:
    def __init__(self,clip_no,stime,etime,weights_data,label):
        self.clip_no = clip_no
        self.stime = stime
        self.etime = etime
        self.weights_data = weights_data
        self.label = label

In [ ]:
#Function to run video input,distribution formation and input to GMM and get updated weights as output
def run(path,ann_file_path):
    mag_arr_all_clips,loa = video_input(path,ann_file_path)
    k,initial_weights,means,cov = intial_parameters(mag_arr_all_clips[0])
    data_object_array=[]
    for index in range(len(mag_arr_all_clips)):
        updated_weights=[]
        updated_means=[]
        updated_variances=[]
        dist_arr = func_distribution_formation(mag_arr_all_clips[index])
        for i in range(dist_arr.shape[0]):
            gmm = GMM(dist_arr[i],k+1,initial_weights.copy(),means.copy(),cov.copy(),50)
            gmm.run()
            updated_weights.append(gmm.weights)
            updated_means.append(gmm.means)
            updated_variances.append(gmm.variances)
        data_object_array.append(data_objects(loa[index].clip_no,loa[index].stime,loa[index].etime,np.asarray(updated_weights).flatten(),loa[index].label))        
    return data_object_array

In [ ]:
#running our entire code.
ann_file_path = "E:\\Study\\Sem Project\\Data\\abnormal_times.txt"
path = "E:\\Study\\Sem Project\\Data\\traffic-junction_3.avi"
total_data_objects = run(path,ann_file_path) #path to video and path to annotations

In [ ]:
def plot_roc_curve(fpr, tpr):
    plt.plot(fpr, tpr, color='orange', label='ROC')
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend()
    plt.show()

In [ ]:
def kernel(X1, X2, l=1.0, sigma_f=1.0):
    sqdist = np.sum(X1**2, 1).reshape(-1, 1) + np.sum(X2**2, 1) - 2 * np.dot(X1, X2.T)
    return sigma_f**2 * np.exp(-0.5 / l**2 * sqdist)

In [ ]:
#Guassian process regression
#Input:X1: Array of m points (m x d),X2: Array of n points (n x d).
#Output: Covariance matrix (m x n).

def posterior_predictive(X_s, X_train, Y_train, l=1.0, sigma_f=1.0, sigma_y=1e-8):
    #equation(6)
    K = kernel(X_train, X_train, l, sigma_f) + sigma_y**2 * np.eye(len(X_train))
    K_s = kernel(X_train, X_s, l, sigma_f)
    K_ss = kernel(X_s, X_s, l, sigma_f) + 1e-8 * np.eye(len(X_s))
    K_inv = inv(K)
    #equation(4)
    mu_s = K_s.T.dot(K_inv).dot(Y_train)
    #equation(5)
    cov_s = K_ss - K_s.T.dot(K_inv).dot(K_s)    
    return mu_s, cov_s

In [ ]:
t_rel = 1 # threshold for q_relative criteria
kern = 1.0*RBF(1.0)
auc=[]
ffpr=[]
ttpr=[]

for j in range(10):
    gpc = GaussianProcessClassifier(kernel=kern,random_state=0)

    #dividing the total dataset into 60% training and 40% test
    total_no = len(total_data_objects)
    print("total objects=",total_no)
    train_no=round(0.6*total_no)
    print("train objects=",train_no)
    test_no=round(0.4*total_no)
    print("test objects=",test_no)

    X_test=[]
    Y_test=[]
    train_objects= []

    #for taking two normal and one abnormal as new train to GPClassifier.
    feature_train = []
    label_train = []
    ab=0
    n=0
    
    np.random.shuffle(total_data_objects)
    for i in range(train_no):
        if total_data_objects[i].label == -1 and n < 2:
            feature_train.append(total_data_objects[i].weights_data)
            label_train.append(total_data_objects[i].label)
            n+=1
        elif total_data_objects[i].label == 1 and ab < 1:
            feature_train.append(total_data_objects[i].weights_data)
            label_train.append(total_data_objects[i].label)
            ab+=1
        else :
            train_objects.append(total_data_objects[i])

    for i in range(train_no,total_no):
        X_test.append(total_data_objects[i].weights_data)
        Y_test.append(total_data_objects[i].label)

    feature_train = np.asarray(feature_train)
    label_train = np.asarray(label_train).reshape(-1,1)
    X_test=np.asarray(X_test)
    Y_test=np.asarray(Y_test)

    print("initial feature length",len(feature_train))
    for val in range(10): 
        q=0
        new_objects = []
        # training the objects
        for i in range(len(train_objects)):
            mu_s,cov_s = posterior_predictive(train_objects[i].weights_data.reshape(1,len(train_objects[i].weights_data)), feature_train, label_train) #giving input to gp regression

            q_rel = min(2*abs(mu_s),2/abs(np.sqrt(cov_s)))  # calculating q rel criteria
            if q_rel < t_rel:    # if q_rel is less than given threshold then give the clip to domain expert
                q+=1
                '''
                print("enter label for clip no: ",train_objects[i].clip_no,"with start time ",train_objects[i].stime," end time ",train_objects[i].etime)
                ll = int(input())   # taking input from user(domain_expert)
                '''

                ll=train_objects[i].label
                feature_train = np.vstack([feature_train,train_objects[i].weights_data]) #adding newly labeled sample to training features
                label_train = np.append(label_train,ll)  # adding new label to training labels
            else:
                new_objects.append(train_objects[i])

        gpc.fit(feature_train,label_train)
        pred_labels=gpc.predict(X_test)
        train_objects.clear()
        print("total no of queries ",q)
        train_objects = new_objects.copy()
        print("feature length",len(feature_train))
        print("train length",len(train_objects))

        #performance
        print("accuracy =",accuracy_score(pred_labels,Y_test)*100)
        print("confusion matrix",confusion_matrix(pred_labels,Y_test))

    auc.append(roc_auc_score(Y_test,pred_labels))
    print('AUC:',auc[j])
    fpr, tpr, thresholds = roc_curve(Y_test,pred_labels)
    ffpr.append(fpr)
    ttpr.append(tpr)
    plot_roc_curve(fpr, tpr)

In [ ]:
print("avg auc score",np.mean(auc))